In [2]:
import astropy.io.fits as fits
import numpy as np
import fitsio, os, sys
from astropy.table import Column
from astropy.table import Table
from math import cos,pi,log10
import matplotlib.pylab as plt
import healpy as hp
from desitarget.geomask import radec_match_to 
#m1, m2, dist = radec_match_to(mtl, sdss[ind], return_sep=True) 
